In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import numpy as np
import pandas as pd


In [ ]:
s = Service("C:\selenium driver\chromedriver")
myoptions = Options()
# myoptions.headless = True
# driver = Chrome(service=s, options=myoptions)

driver = Chrome(service=s)

In [ ]:
# driver.quit()

In [ ]:
# URL of the tripadvisor
url = 'https://www.tripadvisor.com/Hotels'
driver.get(url)

driver.implicitly_wait(10)

In [ ]:
# Xpath of the search box
srch = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[2]/header/div/nav/div/div[1]/div/div/form/input[1]')

In [ ]:
# Enter the destination
srch.send_keys('Pakistan')
srch.submit()
# Wait
driver.implicitly_wait(10)

In [ ]:
#  XPath of the HOTEL only tab
srch = driver.find_element(By.XPATH, '//*[@data-tab-name="Hotels"]')
# click on the HOTEL only tab
srch.click()
# wait
driver.implicitly_wait(10)

In [ ]:
# getting all the hotels in a single page
hotels = driver.find_elements(By.CLASS_NAME, 'result-title')
len(hotels)

In [ ]:
# Extracting all links from the pages you describe

def get_all_links(driver, base_link='https://www.tripadvisor.com', pagination_length=10):
    """
    This function extracts all the links from the pages you describe
    :param driver: your driver
    :param base_link: the base link of the website
    :param pagination_length:  the length of the pagination
    :return: a list of all the links
    """
    all_links = []
    for temp in range(pagination_length):
        page = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT, "Next"))) 
        try:
            # Scrolling down to the reviews Next button
            driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.LINK_TEXT,'Next'))

            all_hotels = driver.find_elements(By.CLASS_NAME, 'result-title')
            links = [os.path.join(base_link, val.get_attribute("onclick").split(",")[3][3:-1]) for val in all_hotels]
            all_links.extend(links)

            # next button
            button = driver.find_element(By.LINK_TEXT, "Next")
            button.click()

            driver.implicitly_wait(10)
        except Exception as e:
            print(e)
            break
    return all_links
    
all_links = get_all_links(driver,base_link='https://www.tripadvisor.com', pagination_length=5)
len(all_links)


In [ ]:
# Extracting reviews from the links

def get_reviews(driver):
    """
    This function extracts the reviews from the given page
    :param driver: your driver 
    :return: a list of reviews
    """
    class_path = driver.find_element(By.CLASS_NAME, 'cRVSd')

    # Date on the review was given
    if class_path.find_element(By.XPATH, "./span").text.split(' ') [-1] == "Yesterday":
        review_date = "Aug 2022"
    else:
        review_date = ' '.join(class_path.find_element(By.XPATH, "./span").text.split(' ')[-2:])

    # Username and Profile Link on TripAdvisor of the reviewer
    username = class_path.find_element(By.CLASS_NAME, "ui_header_link.uyyBf").text
    profile_link = class_path.find_element(By.CLASS_NAME, "ui_header_link.uyyBf").get_attribute("href")

    # Global Address of the reviewer
    class_path = driver.find_element(By.CLASS_NAME, 'MziKN')
    try:
        person_address = class_path.find_element(By.CLASS_NAME, 'default.LXUOn.small').text
    except:
        person_address = np.NaN
    
    try:
        # Rating given in the review
        review_rating = int(driver.find_element(By.XPATH, './div[@class="WAllg _T"]/div/div/span').get_attribute('class').split("_")[3])/10
    except:
        print("Exception in review rating")

    # Review Title
    title = driver.find_element(By.XPATH, './div[@class="WAllg _T"]/div/a/span/span').text

    # Review Text
    review_text = driver.find_element(By.XPATH, "./div[@class=\"WAllg _T\"]/div/div/div/q/span").text

    # Date of the stay in the hotel
    stay_date = ' '.join(driver.find_element(By.XPATH, './div[@class=\"WAllg _T\"]//*/span[@class="teHYY _R Me S4 H3"]').text.split(' ')[-2:])
    
    return username, profile_link, person_address, stay_date, review_date, review_rating, title, review_text


def get_reviews_with_pagination(driver, link, pagination_length=10):
    """
    This function extracts the reviews from the links of different hotels and saves them as csv
    :param driver: your driver
    :param links: the link of the hotel
    :return: Nothing
    """
    file_name = "TripAdvisor.csv"

    # Opening the link
    print("Opening the link...")
    driver.get(link)
    print("Link opened successfully.")
    driver.implicitly_wait(10)
    # Hotel Name
    hotel_name = driver.find_element(By.ID, "HEADING").text

    print("Hotel Name : ",hotel_name)
    # Hotel Rating respective to the city
    ranking = driver.find_element(By.XPATH, "//div[@class=\"cGAqf\"]/a/span").text

    # Hotel Global Address
    global_address = driver.find_element(By.XPATH, '//span[@class="fHvkI PTrfg"]').text
    
    for temp in range(pagination_length):
        df = pd.DataFrame(columns=['username', 'profile_link', 'person_address', 'hotel_name', 'hotel_ranking', 'hotel_address',
                                    'stay_date', 'review_date', 'review_rating', 'title', 'review_text', 'hotel_link'])
                                    
        page = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT,'Next'))) 

        try:
            # Scrolling down to the reviews Next button
            # driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.LINK_TEXT,'Next'))
            
            # All the reviews present on the current page
            reviews = driver.find_elements(By.XPATH, '//div[@class="YibKl MC R2 Gi z Z BB pBbQr"]')
            driver.implicitly_wait(10)
            count = 0 
            for review in reviews:
                try:
                    username, profile_link, person_address, stay_date, review_date, review_rating, title, review_text = get_reviews(review)
                except Exception as e:
                    reviews = driver.find_elements(By.XPATH, '//div[@class="YibKl MC R2 Gi z Z BB pBbQr"]')
                    count += 1
                    continue
                # saving in the dataframe
                df.loc[len(df)] = [username, profile_link, person_address, hotel_name, ranking, global_address, 
                                    stay_date, review_date, review_rating, title, review_text, link]

            print("Total Reviews done = ", (temp + 1) * 10 - count)
            # saving files to csv
            if os.path.exists(file_name):
                df.to_csv(file_name, mode='a', header=False, index=False) 
            else:            
                df.to_csv("TripAdvisor.csv", index=False)

            # Clicking on the next page of reviews
            
            try:
                button = driver.find_element(By.LINK_TEXT, "Next")
                button.click()
            except:
                print("Exception in button click.")
                button = driver.find_element(By.LINK_TEXT, "Next")
                button.click()

            # waiting
            driver.implicitly_wait(10)
        except Exception as e:
            print("Exception in pagination. Moving to next hotel : ", e)
            break
    
        
# Looping through links
for link in all_links:
    get_reviews_with_pagination(driver, link, pagination_length=2)